In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import time
from pandas import ExcelWriter
from pandas import ExcelFile
from pprint import pprint
from scipy.stats import linregress
from datetime import timedelta

In [24]:
fileordprod = "./data/brazilian-ecommerce/olist_order_items_dataset.csv"
fileordcusttime = "./data/brazilian-ecommerce/olist_orders_dataset.csv"
fileprodcat = "./data/brazilian-ecommerce/olist_products_dataset.csv"
fileprodtransl = "./data/brazilian-ecommerce/product_category_name_translation.csv"
filecustzip = "./data/brazilian-ecommerce/olist_customers_dataset.csv"
filezipcoord = "./data/brazilian-ecommerce/olist_geolocation_dataset.csv"

# Read Purchasing File and store into Pandas data frame
ordprod = pd.read_csv(fileordprod)#[['order_id', 'product_id', 'price']]
ordcusttime= pd.read_csv(fileordcusttime)#[['order_id', 'customer_id', 'order_purchase_timestamp']]
prodcat = pd.read_csv(fileprodcat)#[['product_id', 'product_category_name']]
prodtransl = pd.read_csv(fileprodtransl)
custzip = pd.read_csv(filecustzip)[['customer_id','customer_zip_code_prefix']]
zipcoord = pd.read_csv(filezipcoord)[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]

zipcoord = zipcoord.rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix'}).drop_duplicates(subset = 'customer_zip_code_prefix')




mergedf = pd.merge(ordcusttime, ordprod, on = 'order_id',how= 'left')
mergedf = pd.merge(mergedf, prodcat, on = 'product_id', how = 'left')
mergedf = pd.merge(mergedf, prodtransl, on = 'product_category_name', how = 'left')
mergedf = pd.merge(mergedf, custzip, on = 'customer_id', how = 'left')
mergedf = pd.merge(mergedf, zipcoord, on = 'customer_zip_code_prefix' , how = 'left')

renamecolumns = {
    'order_purchase_timestamp': 'order_time',
    'product_category_name_english': 'category'
}
mergedf = mergedf.rename(columns = renamecolumns)


#minus one day
mergedf["date_time"]= pd.to_datetime(mergedf['order_time']) - timedelta(days=1)

mergedf['city'] = ''
mergedf['citylon'] = ''
mergedf['citylat'] = ''

#converted to MM/YY Format
mergedf['YYYY'] = pd.DatetimeIndex(mergedf['date_time']).year
mergedf['MM'] = pd.DatetimeIndex(mergedf['date_time']).month

mergedf['month'] = pd.to_datetime(mergedf['date_time'])
mergedf['day'] = mergedf['month'].dt.strftime('%m/%d/%y')
mergedf['month'] = mergedf['month'].dt.strftime('%m/%y')
print(mergedf.shape)
mergedf.head()

(113425, 34)


,order_id,customer_id,order_status,order_time,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,geolocation_lat,geolocation_lng,date_time,city,citylon,citylat,YYYY,MM,month,day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,-23.574809,-46.587471,2017-10-01 10:56:33,,,,2017,10,10/17,10/01/17
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,...,-12.169860,-44.988369,2018-07-23 20:41:37,,,,2018,7,07/18,07/23/18
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,...,-16.746337,-48.514624,2018-08-07 08:38:49,,,,2018,8,08/18,08/07/18
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,-5.767733,-35.275467,2017-11-17 19:28:06,,,,2017,11,11/17,11/17/17
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,-23.675037,-46.524784,2018-02-12 21:18:39,,,,2018,2,02/18,02/12/18


In [23]:
# weatherurl = 'http://api.openweathermap.org/data/2.5/weather?'
# weatherapikey = 'b457821be7196c46ce037583b6f2feb4'
# for index, row in mergedf.iterrows():
#     latitude = mergedf.loc[mergedf.index[index],'geolocation_lat']
#     longitude = mergedf.loc[mergedf.index[index],'geolocation_lng']
    
#     params = {
#     'lon': longitude,
#     'lat': latitude,
#     'appid': weatherapikey
#     }
    
#     cityresponse = requests.get(weatherurl, params = params).json()
    
#     mergedf.loc[index, 'city'] = cityresponse['name']
    
#     if index % 500 == 0:
#         print(index, 'city')

# weatherurl = 'http://api.openweathermap.org/data/2.5/weather?'
# weatherapikey = 'b457821be7196c46ce037583b6f2feb4'
# for index, row in mergedf.iterrows():
#     city = mergedf.loc[mergedf.index[index],'city']
    
#     params = {
#     'q': city,
#     'appid': weatherapikey
#     }
    
#     cityresponse = requests.get(weatherurl, params = params).json()
#     mergedf.loc[index, 'citylon'] = cityresponse['coord']['lon']
#     mergedf.loc[index, 'citylat'] = cityresponse['coord']['lat']
#     if index % 500 == 0:
#         print(index, 'citylon & citylat')
    
# mergedf.to_csv('MergeDF.csv')


In [49]:
print(min(mergedf['geolocation_lng']) - max(mergedf['geolocation_lng']))
print(min(mergedf['geolocation_lat']))
print(max(mergedf['geolocation_lat']))
np.arange(min(mergedf['geolocation_lng']), max(mergedf['geolocation_lng']))
np.arange(min(mergedf['geolocation_lat']), max(mergedf['geolocation_lat']))

-64.08885053080984
-36.6053744107061
42.18400274298598


array([-36.60537441, -35.60537441, -34.60537441, -33.60537441,
       -32.60537441, -31.60537441, -30.60537441, -29.60537441,
       -28.60537441, -27.60537441, -26.60537441, -25.60537441,
       -24.60537441, -23.60537441, -22.60537441, -21.60537441,
       -20.60537441, -19.60537441, -18.60537441, -17.60537441,
       -16.60537441, -15.60537441, -14.60537441, -13.60537441,
       -12.60537441, -11.60537441, -10.60537441,  -9.60537441,
        -8.60537441,  -7.60537441,  -6.60537441,  -5.60537441,
        -4.60537441,  -3.60537441,  -2.60537441,  -1.60537441,
        -0.60537441,   0.39462559,   1.39462559,   2.39462559,
         3.39462559,   4.39462559,   5.39462559,   6.39462559,
         7.39462559,   8.39462559,   9.39462559,  10.39462559,
        11.39462559,  12.39462559,  13.39462559,  14.39462559,
        15.39462559,  16.39462559,  17.39462559,  18.39462559,
        19.39462559,  20.39462559,  21.39462559,  22.39462559,
        23.39462559,  24.39462559,  25.39462559,  26.39

In [39]:
stepsize = 1
for x in range(0, 10, stepsize):
    for y in range(0, 10, stepsize):
        print(x,y)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9


In [32]:
range?


In [ ]:
urlBOV = "https://www.quandl.com/api/v3/datasets/BCB/7.json?api_key=2vRuu5GNchksqBdcAHzc"
responseBOV = requests.get(urlBOV).json()


In [ ]:
days = 10

BOV_df = pd.DataFrame(responseBOV['dataset']['data'])

BOV_df = BOV_df.rename(index=str, columns={0: "day", 1: "BOV"})

BOV_df['day'] = pd.to_datetime(BOV_df['day'])
BOV_df['day'] = BOV_df['day'].dt.strftime('%m/%d/%y')
BOV_df['BOV'] = pd.to_numeric(BOV_df['BOV'])
BOV_df = BOV_df.sort_values(by = 'day', ascending = True)
BOV_df['BOVdiff'] = BOV_df['BOV'].diff(periods = days)
BOV_df['BOVma'] = BOV_df['BOV'].rolling(window = days).mean()
BOV_df.head(20)

metriclist = ['BOV', 'BOVdiff', 'BOVma']
mergedf = pd.merge(mergedf, BOV_df, on = 'day', how = 'left')

In [ ]:
# mergedf['day'] = pd.to_datetime(mergedf['day']) - 1


In [ ]:
#mergedf['category'].unique()

In [ ]:
# red = fashion/clothing
    # 'perfumery', 'watches_gifts','fashion_underwear_beach','fashion_bags_accessories','fashion_shoes','fashion_male_clothing','fashion_sport','fashio_female_clothing','fashion_childrens_clothes'
# orange = home/furniture
    # 'housewares','furniture_decor','office_furniture','bed_bath_table','home_confort','furniture_bedroom','kitchen_dining_laundry_garden_furniture','home_comfort_2','furniture_mattress_and_upholstery'
# yellow = construction/tools
    # 'garden_tools','construction_tools_construction','construction_tools_lights','construction_tools_garden','construction_tools_safety','home_construction','construction_tools_tools'
# purple = electronics
    # 'computers_accessories','telephony','electionics','cds_dvds_musicals','consoles_games','audio','tablets_printing_image','computers','dvds_blu_ray','music'
# green = food and drink
    # 'food','drinks','food_drink','la_cuisine'
# cyan = books
    # 'books_general_interest','books_imported','books_technical'
# gray = appliances
    # 'air_conditioning','small_appliances','home_appliances','home_appliances_2','small_appliances_home_oven_and_coffee'

In [ ]:
new_categories = {'fashion_clothing': ['perfumery', 'luggage_accessories','watches_gifts','fashion_underwear_beach','fashion_bags_accessories','fashion_shoes','fashion_male_clothing','fashion_sport','fashio_female_clothing','fashion_childrens_clothes'],
                  'home_furniture': ['housewares','art','arts_and_craftmanship','furniture_decor','office_furniture','bed_bath_table','home_confort','furniture_bedroom','kitchen_dining_laundry_garden_furniture','home_comfort_2','furniture_mattress_and_upholstery'],
                  'construction_tools': ['garden_tools','construction_tools_construction','construction_tools_lights','construction_tools_garden','construction_tools_safety','home_construction','construction_tools_tools'],
                  'electronics': ['computers_accessories','telephony','electionics','cds_dvds_musicals','consoles_games','audio','tablets_printing_image','computers','dvds_blu_ray','music'],
                  'food_drink': ['food','drinks','food_drink','la_cuisine'],
                  'books': ['books_general_interest','books_imported','books_technical'],
                  'appliances': ['air_conditioning','small_appliances','home_appliances','home_appliances_2','small_appliances_home_oven_and_coffee'],
                    'auto': ['auto'],
                    'industry':['agro_industry_and_commerce','industry_commerce_and_business'],
                    'other':['pet_shop','stationary','toys','baby','cool_stuff','sports_leisure','diapers_and_hygeine','signaling_and_security','party_supplies','cine_photo','market_place','musical_instruments','christmas_supplies']}
                    
new_dict = {}
for key,valuelist in new_categories.items():
    for v in valuelist:
        new_dict[v] = key
print(new_dict)


In [ ]:
mergedf['category'] = mergedf.category.replace(new_dict)

In [ ]:
#Rid of black friday
mergedf = mergedf.loc[mergedf['day'] != '11/24/17']

In [ ]:
#commented out because it lags when rerunning all 

#save to excel file 

# writer = ExcelWriter('output.xlsx')
# mergedf.to_excel(writer,'Sheet1',index=False)
# writer.save()

In [ ]:
home_furniture = mergedf['price'][mergedf['category']=='home_furniture'].sum()
fashion_clothing = mergedf['price'][mergedf['category']=='fashion_clothing'].sum()
construction_tools = mergedf['price'][mergedf['category']=='construction_tools'].sum()
electronics = mergedf['price'][mergedf['category']=='electronics'].sum()
food_drink = mergedf['price'][mergedf['category']=='food_drink'].sum()
books = mergedf['price'][mergedf['category']=='books'].sum()
appliances = mergedf['price'][mergedf['category']=='appliances'].sum()
#data['duration'][data['item'] == 'call'].sum()

total_spend = home_furniture + fashion_clothing + construction_tools + electronics + food_drink + books + appliances

category_spend = {home_furniture:'home_furniture',
                        fashion_clothing:'fashion_clothing',
                        construction_tools:'construction_tools',
                        electronics:'electronics',
                        food_drink:'food_drink',
                        books:'books',
                        appliances:'appliances'
                       }

In [ ]:
mergedf['category'].nunique()

In [ ]:
print(category_spend)

In [ ]:
print(total_spend)

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'home/furniture', 'fashion/clothing', 'construction/tools', 'electronics','food and drink','books','appliances'
sizes = [home_furniture, fashion_clothing, construction_tools, electronics, food_drink, books, appliances]
explode = (0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
       shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('total spend')
plt.show()

In [ ]:
#Black Friday!
mergedf.loc[mergedf['day'] == '11/24/17']

In [ ]:
groupdf = mergedf.groupby(['BOVdiff', 'category'])
groupdf = groupdf.sum()
groupdf.head()

In [ ]:
#BY CATEGORIES
for metric in metriclist:
    for cat in new_categories.keys():
        groupdf = mergedf.groupby([metric, 'category'])
        groupdf = groupdf.sum()
        #sum of prices
        #groupdf = groupdf.sum()
        groupdf.reset_index(inplace = True)
        #why are there sum of prices at 0? no data from those dates?
        groupdf = groupdf.loc[groupdf['price'] > 1]
        #change categories
        groupdf = groupdf.loc[groupdf['category'] == cat]

        plt.scatter(groupdf[metric], groupdf['price'], color=["coral"], edgecolor="black", alpha = 0.75,marker="o")
        plt.ylim(0, max(groupdf['price']))
        (slopeBOV, interceptBOV, rBOV, _, _) = linregress(groupdf[metric], groupdf['price'])
        fitBOV = slopeBOV * groupdf[metric] + interceptBOV
        plt.ylabel('Revenue per Day')
        plt.xlabel(metric)
        plt.title(f'Revenue vs {metric} for {cat}')
        print(type(metric))
        plt.plot(groupdf[metric], fitBOV)
        print(f'R2 = {rBOV**2}')
        plt.savefig('./Graphs/{metric}{cat}.png')
        plt.show()

In [ ]:
#ALL CATEGORIES
for metric in metriclist:
    groupdf = mergedf.groupby(metric)
    groupdf = groupdf.sum()
    groupdf.reset_index(inplace = True)
    #why are there sum of prices at 0? no data from those dates?
    groupdf = groupdf.loc[groupdf['price'] > 1]


    plt.scatter(groupdf[metric], groupdf['price'])
    plt.ylim(0, max(groupdf['price']))
    (slopeBOV, interceptBOV, rBOV, _, _) = linregress(groupdf[metric], groupdf['price'])

    fitBOV = slopeBOV * groupdf[metric] + interceptBOV
    print(metric, rBOV**2)
    plt.plot(groupdf[metric], fitBOV)
    plt.show()
    

In [ ]:
#used to see what the outlier point was
#groupdf.sort_values('price', ascending= False).head()